In [51]:
# import ase modules
import ase
import ase.io
import ase.build
import ase.visualize

# import numpy and other modules
import os
import numpy as np

In [46]:

Pb = ase.Atom(position=(0.5,0.5,0.5), symbol='Pb')
O1 = ase.Atom(position=(0.5,0.5,0.0), symbol='O')
O2 = ase.Atom(position=(0.0,0.5,0.5), symbol='O')
O3 = ase.Atom(position=(0.5,0.0,0.5), symbol='O')
Ti = ase.Atom(position=(0.0,0.0,0.0), symbol='Ti')

atoms = ase.Atoms((Pb,O1,O2,O3,Ti), cell=[1.0,1.0,1.0])
cell = atoms.get_cell()

cell *= 5.0

atoms.set_cell(cell,scale_atoms = True)
ase.visualize.view(atoms)



<Popen: returncode: None args: ['c:\\Users\\skraw\\AppData\\Local\\Programs\...>

In [47]:
positions = {'Pb': (0.5, 0.5, 0.5),
             'O1': (0.5, 0.5, 0.0),
             'O2': (0.0, 0.5, 0.5),
             'O3': (0.5, 0.0, 0.5),
             'Ti': (0.0, 0.0, 0.0)}

# Create Atoms object
atoms = ase.Atoms(symbols=['Pb', 'O', 'O', 'O', 'Ti'],
              scaled_positions=[positions['Pb'],
                         positions['O1'],
                         positions['O2'],
                         positions['O3'],
                         positions['Ti']],
              cell=[5, 5, 5])

cell = atoms.get_cell()
atoms.set_cell(cell,scale_atoms = True)

cell *= 5.0

# Save Atoms object as POSCAR file
ase.visualize.view(atoms)

ase.io.write('pb_poscar',atoms,format="vasp")

In [54]:
positions = {'Sr': (0.5, 0.5, 0.5),
             'O1': (0.5, 0.5, 0.0),
             'O2': (0.0, 0.5, 0.5),
             'O3': (0.5, 0.0, 0.5),
             'Ti': (0.0, 0.0, 0.0)}

# Create Atoms object
atoms = ase.Atoms(symbols=['Sr', 'O', 'O', 'O', 'Ti'],
              scaled_positions=[positions['Sr'],
                         positions['O1'],
                         positions['O2'],
                         positions['O3'],
                         positions['Ti']],
              cell=[5, 5, 5])

cell = atoms.get_cell()
atoms.set_cell(cell,scale_atoms = True)


# Save Atoms object as POSCAR file
ase.visualize.view(atoms)

ase.io.write('sr_poscar',atoms,format="vasp")

In [50]:
# Load bulk structures of PbTiO3 and SrTiO3
pbtio3 = ase.io.read('pb_poscar')
srtio3 = ase.io.read('sr_poscar')
cell = pbtio3.get_cell()
z_pbtio3 = cell[2][2]

# Repeat the unit cells to desired thickness
pbtio3_repeated = pbtio3.repeat((1, 1, 2))
srtio3.translate((0,0,2*z_pbtio3))  # 2 unit cells along the z-direction
srtio3_repeated = srtio3.repeat((1, 1, 3))  # 3 unit cells along the z-direction

# Combine the two structures along the z-direction
superlattice = pbtio3_repeated + srtio3_repeated

ase.visualize.view(superlattice)


ase.io.write('superlattice_POSCAR', superlattice, format='vasp')

In [55]:
pbtio3_unit_cell = ase.io.read('superlattice_POSCAR')

# Create a 4x4x4 supercell of PbTiO3
pbtio3_supercell = pbtio3_unit_cell.repeat((4, 4, 4))

# Get the list of Pb atoms that are not on the boundary
non_boundary_pb_atoms = [atom for atom in pbtio3_supercell if not any([np.isclose(atom.position[i], 0.0) or np.isclose(atom.position[i], pbtio3_supercell.cell[i, i]) for i in range(3)])]

# Choose a random Pb atom and its neighboring Ti atom
random_pb_atom = np.random.choice(non_boundary_pb_atoms)
neighbor_ti_atom = [atom for atom in pbtio3_supercell if np.linalg.norm(atom.position - random_pb_atom.position) < 1.2 and atom.symbol == 'Ti'][0]

# Swap positions to create an antisite Pb-Ti defect
random_pb_atom.symbol, neighbor_ti_atom.symbol = neighbor_ti_atom.symbol, random_pb_atom.symbol

# Save the structure with the antisite defect as a POSCAR file
ase.visualize.view(pbtio3_supercell)

IndexError: list index out of range